In [23]:
import networkx as nx
import numpy as np
import random
import matplotlib.pyplot as plt
import re
import pickle

In [24]:
N = 100
chromatic_num = 2     #this one only works for n = 2

In [25]:
def load_list_of_dicts(filename, create_using=nx.Graph):
    with open(filename, 'rb') as f:
        list_of_dicts = pickle.load(f)
    graphs = [create_using(graph) for graph in list_of_dicts]
    return graphs

# Change the path to your local path

In [26]:
path = "drive/MyDrive/Graphs/k-2_100-graphs_400-vertices.pkl"

In [27]:
filename = path.split('/')[-1]
pattern = r"k-(\d+)_[\w-]+_(\d+)-vertices\.pkl"
match = re.search(pattern, filename)

if match:
    chromatic_num = int(match.group(1))
    n = int(match.group(2))
    print(f"chromatic_num={chromatic_num}, n={n}")
else:
    print("Filename does not match pattern")

chromatic_num=2, n=400


# The CBIP Algorithm and test

In [28]:
def get_bipartite_independent_sets(graph):
    independent_set_1 = set()
    independent_set_2 = set()
    visited = set()

    # choose the last vertex and add it to independent set 1
    # start_vertex = list(graph.nodes())[-1]
    start_vertex = len(list(graph.nodes())) - 1
    independent_set_1.add(start_vertex)
    visited.add(start_vertex)

    # perform a breadth-first search(BFS) to add vertices to independent sets
    queue = [start_vertex]
    while queue:
        vertex = queue.pop(0)
        for neighbor in graph[vertex]:
            if neighbor not in visited:
                # add neighbor to the opposite independent set
                if vertex in independent_set_1:
                    independent_set_2.add(neighbor)
                else:
                    independent_set_1.add(neighbor)
                visited.add(neighbor)
                queue.append(neighbor)

    # return the two independent sets
    return list(independent_set_1), list(independent_set_2)


In [29]:
def find_minimum_integer_not_in_list(lst):
    s = set(lst)
    i = 1
    while i in s:
        i += 1
    return i

In [30]:
def cpib(G):
  n = len(G.nodes())
  if n<1:
    return []
  #Color the first vertice 1 as our base case.
  color_list = [1]
  for i in range(1,n):
    other_part_colors = []
    other_part = get_bipartite_independent_sets(G.subgraph(range(i+1)))[1]
    for j in other_part:
      other_part_colors.append(color_list[j])
    color = find_minimum_integer_not_in_list(other_part_colors)
    color_list.append(color)
  return color_list


In [31]:
all_graphs = load_list_of_dicts(path)

In [32]:
# all_graphs = g_generator(N, n, chromatic_num)     #change g_generator with his code

In [33]:
ratio_list_cbip = []
for i in range(N):
  # print(i)
  G = all_graphs[i]
  ratio_list_cbip.append(max(cpib(G))/chromatic_num)
  ##Write it into a file for large graphs
  # with open('my_list.txt', 'w') as f:
  #   f.write(','.join(str(x) for x in ratio_list_cbip))

In [34]:
print("nember of vertices:", n)
print("The maximum ratio is:",max(ratio_list_cbip))
print("The ratio's mean is :",np.mean(np.array(ratio_list_cbip)),"And the standard deviation is:",np.std(np.array(ratio_list_cbip)))

nember of vertices: 400
The maximum ratio is: 2.0
The ratio's mean is : 1.425 And the standard deviation is: 0.29474565306378986
